# fdtd_python

> A Python package for Finite-Difference Time-Domain (FDTD) simulations.

In [ ]:
#| export 
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt

In [ ]:
#| default_exp fdtd_python

In [ ]:
#| export
class fdtd_settings:
    def __init__(self,
                 simulation_size, # distance in microns (float)
                 simulation_time, # time in ps (#float)
                 thicknesses, # array
                 refractive_indices): # array
        
        self.simulation_size = simulation_size
        self.simulation_time = simulation_time
        self.thicknesses = thicknesses
        self.refractive_indices = refractive_indices

In [ ]:
#| export
def easeInOutQuad(t_in):
    L = len(t_in)
    tout = np.zeros(L)
    t_final = t_in[-1]
    t_dummy = t_in/t_final
    for i,tval in enumerate(t_dummy):
        t = tval.copy()
        t*=2
        if t < 1:
            dummy = t * t / 2
        else:
            t-=1
            dummy = -((t-2)*t - 1) / 2
        tout[i] = dummy
    return tout*t_final

In [ ]:
#| export 
def fdtd_run(wavelength, N_w, settings):
    """
    wavelength is in microns
    Let's have a go at making this work with units!!!
    N_w: points per wavelength
    """
    c = 300/wavelength # microns/ps
    Courant = 1
    dx = 1/N_w # microns
    
    """
    """
    dt = dx/c
    t_final = settings.simulation_time # ps
    x_final = settings.simulation_size/wavelength # microns
    
    t_arr = np.arange(0,t_final,dt)
    x_arr = np.arange(0,x_final,dx)

    tt = np.linspace(0,len(t_arr),len(t_arr))
    t_arr_eased = easeInOutQuad(tt)

    # fig, ax = plt.subplots()
    # dt = t_arr[1]-t_arr[0]
    # ax.plot([int(t) for t in t_arr_eased if (int(t) % 10)==0])
    
    t_size = len(t_arr)
    x_size = len(x_arr)
    
    ez = np.zeros(x_size)
    hy = np.zeros(x_size)
    
    imp0 = 377.0/wavelength
    LOSS = 0.001
    epsR = np.zeros(x_size)
    ceze = np.zeros(x_size)
    cezh = np.zeros(x_size)

    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
    left_edge = buffer_size/wavelength
    right_edge = left_edge + np.sum(settings.thicknesses)/wavelength

    lnew = np.zeros((len(settings.thicknesses),2))

    colors = ['blue','red','green']
    buffer_size = (settings.simulation_size - np.sum(settings.thicknesses))/2
        
    for i,x in enumerate(x_arr):

        if 0<=x<left_edge or x>=right_edge:
            epsR[i] = 1.0
            ceze[i] = 1.0
            cezh[i] = imp0
            
        for j,d in enumerate(settings.thicknesses):
            left_edge_new = left_edge + np.sum(settings.thicknesses[:j])/wavelength
            right_edge_new = left_edge + np.sum(settings.thicknesses[:j+1])/wavelength            
            lnew[j,:] = [left_edge_new,right_edge_new]
            # print(left_edge, right_edge)
            if left_edge_new<x<=right_edge_new:
                 epsR[i] = settings.refractive_indices[j]
                 ceze[i] = (1.0 - LOSS) / (1.0 + LOSS)
                 cezh[i] = imp0 / epsR[i] / (1.0 + LOSS)
            # print(epsR[i])
    #print(epsR)
    e_50 = np.zeros(t_size) # Efield at position 50

    fig, ax = plt.subplots()
    
    for i in range(np.size(lnew,0)):
        ax.fill_between((np.array(lnew[i,:])-buffer_size/wavelength)*wavelength,y1=-1.2, y2=1.2,alpha=0.1,color=colors[i])

    ax.set_ylim([-1.2,1.2])
    ax.set_xlabel(r'distance ($\mathrm{\mu}$m)')
    ax.set_ylabel('E(t) (norm.)')

    x_vals = [left_edge,right_edge]
    y_vals = [-1.2,1.2]
    artists = []
    offno = 4
    # ax.vlines(x_vals,y_vals[0],y_vals[1],color='red',alpha=0.5)
    # ax.hlines(y=y_vals,xmin=left_edge,xmax=right_edge,color='red',alpha=0.5)

    # ax.fill_between(x=x_vals, y1=-1, y2=1,alpha=0.1,color='red')

    E_total = np.zeros((len(x_arr),len(t_arr)))
    
    for i,t in enumerate(t_arr):
        
        hy[x_size-1] = hy[x_size-2]
        for j in range(x_size-1):
            hy[j] = hy[j] + (ez[j+1] - ez[j])/imp0

        hy[9] -= np.exp(-((t - offno)/1.0)**2) * np.sin(2*np.pi*c * (t - offno) ) / imp0
        
        ez[0] = ez[1]
        for j in range(1,x_size):
            ez[j] = ceze[j]*ez[j] + cezh[j]*(hy[j]-hy[j-1])

        ez[10] += np.exp(-((t - offno)/.7)**2)*np.sin(2*np.pi*c * (t - offno))
        
        e_50[i] = ez[len(ez)-1]
        
        E_total[:,i] = ez

        nos_frame = len(t_arr)//100

        if (i % nos_frame == 0):
            container = ax.plot(x_arr[10:]*wavelength-buffer_size,ez[10:],color='black')
            
            txt = ax.text(0.45,1.01,'{num:{fill}{width}} ps'.format(num=round(t_arr[i],1), fill='0', width=2),transform=ax.transAxes,fontsize="large")
        
            container.append(txt)
            # txt = plt.text(0.1,1,i)
            #container += ax2.plot(x_arr,h_snaps[:,i],color='green')
            artists.append(container)

    dpi = 100
    writer = animation.writers['pillow'](fps=30)
    #ani.save('test.mp4',writer=writer,dpi=dpi)
    
    ani = animation.ArtistAnimation(fig=fig, artists=artists)
    plt.show()
    ani.save(filename="pillow_example.gif", writer=writer, dpi=dpi)

    return x_arr[10:]*wavelength-buffer_size,t_arr,E_total[10:,:]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()